In [100]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output


In [17]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import os
from multiprocessing import Pool
from loky import get_reusable_executor

# Load templates from specified folders
def load_templates(template_folder, suffix):
    templates = []
    for filename in os.listdir(template_folder):
        if filename.endswith(suffix):
            path = os.path.join(template_folder, filename)
            template = cv2.imread(path, 0)
            templates.append(template)
    return templates

def draw_rectangles(frame, matched_points):
    for (x, y, w, h) in matched_points:
        cv2.rectangle(frame, (x - w // 2, y - h // 2), (x + w // 2, y + h // 2), (255, 0, 0), 2)
    return frame

# Function to get the initial socket positions from the first frame
def get_initial_socket_positions(frame, pocket_template_folder, suffix='.png'):
    pocket_templates = load_templates(pocket_template_folder, suffix)
    pocket_positions = match_templates(frame, pocket_templates, 0.85)
    return pocket_positions


# Match templates
def match_templates(frame, templates, threshold=0.8):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    matched_points = []
    for template in templates:
        w, h = template.shape[::-1]
        res = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
        loc = np.where(res >= threshold)
        for pt in zip(*loc[::-1]):  # Reverse tuple to get (x, y) coordinates
            center_point = (pt[0] + w // 2, pt[1] + h // 2, w, h)  # Include dimensions for drawing
            matched_points.append(center_point)
    return matched_points

def get_socket_positions_from_first_frame(frames, pocket_template_folder):
    if not frames:
        return None, []  # Return None if there are no frames
    
    first_frame = frames[0]
    pocket_templates = load_templates(pocket_template_folder, '.png')
    pocket_positions = match_templates(first_frame, pocket_templates, 0.85)
    return first_frame, pocket_positions

def calculate_middle_sockets(pocket_positions):
    # Assuming pocket_positions have been sorted by their x-coordinate or are in a fixed known order
    leftmost, rightmost = pocket_positions[0], pocket_positions[-1]
    middle_top = ((leftmost[0] + rightmost[0]) // 2, min(leftmost[1], rightmost[1]))
    middle_bottom = ((leftmost[0] + rightmost[0]) // 2, max(leftmost[1], rightmost[1]))
    return middle_top, middle_bottom


def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    i = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if i < 100:
            i += 1
            continue
        elif i > 150:
            break
        frames.append(frame)
        i += 1
        # print(len(frames))
    cap.release()
    return frames

def draw_text_with_background(image, text, font_scale=0.8, thickness=2, max_width=1000):
    font = cv2.FONT_HERSHEY_COMPLEX  # More complex font for better readability
    text_color = (255, 255, 255)  # White text
    bg_color = (0, 0, 255)  # Red background for high visibility

    # Break the text into multiple lines if it's too long
    words = text.split()
    lines = ['']
    for word in words:
        if cv2.getTextSize(lines[-1] + word, font, font_scale, thickness)[0][0] > max_width:
            lines.append(word)
        else:
            lines[-1] += ' ' + word

    y_offset = 30  # Start slightly lower to avoid being too close to the top edge
    for line in lines:
        (line_width, line_height), baseline = cv2.getTextSize(line, font, font_scale, thickness)
        cv2.rectangle(image, (10, y_offset - line_height), (10 + line_width, y_offset + baseline), bg_color, cv2.FILLED)
        cv2.putText(image, line, (10, y_offset), font, font_scale, text_color, thickness, cv2.LINE_AA)
        y_offset += line_height + baseline

    return image





def process_frame(frame, i, ball_template_folder, socket_positions):
    ball_templates = load_templates(ball_template_folder, '.png')
    # pocket_templates = load_templates(pocket_template_folder, '.png')
    
    # Detect balls and pockets in the frame
    ball_positions = match_templates(frame, ball_templates, 0.9)
    # pocket_positions = match_templates(frame, pocket_templates, 0.85)
    
    # Draw on frames
    frame_with_balls = draw_rectangles(frame, ball_positions)
    frame_with_pockets = draw_rectangles(frame_with_balls, socket_positions)
    for socket_pos in socket_positions:
        x, y, w, h = socket_pos
        cv2.rectangle(frame_with_balls, (x - w // 2, y - h // 2), (x + w // 2, y + h // 2), (255, 0, 0), 2)

    # if len(pocket_positions) >= 4:
    #     # Draw markers on middle top and bottom locations between pockets
    #     middle_top, middle_bottom = calculate_middle_sockets(pocket_positions)
    #     cv2.circle(frame_with_pockets, middle_top, 10, (0, 255, 255), -1)
    #     cv2.circle(frame_with_pockets, middle_bottom, 10, (0, 255, 255), -1)
    #     cv2.rectangle(frame_with_pockets, (middle_top[0]-50, middle_top[1]-50), (middle_top[0]+50, middle_top[1]+50), (255, 0, 0), 2)
    #     cv2.rectangle(frame_with_pockets, (middle_bottom[0]-50, middle_bottom[1]-50), (middle_bottom[0]+50, middle_bottom[1]+50), (255, 0, 0), 2)
    #     print('pockets', i)
    #     # Check proximity of each ball to each detected pocket
    #     for ball_pos in ball_positions:
    #         for pocket_pos in pocket_positions:
    #             distance = np.linalg.norm(np.array(ball_pos) - np.array(pocket_pos))
    #             # print("HALO")
    #             if distance < 50:  # Consider this threshold to determine "in the socket"
    #                 text_position = (ball_pos[0], ball_pos[1] - 10)
    #                 cv2.putText(frame_with_pockets, "Ball in the socket", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    #                 print("Ball detected in the socket at position:", ball_pos, i)
    if len(socket_positions) >= 2:
            middle_top, middle_bottom = calculate_middle_sockets(socket_positions)
            cv2.circle(frame_with_pockets, middle_top, 10, (0, 255, 255), -1)
            cv2.circle(frame_with_pockets, middle_bottom, 10, (0, 255, 255), -1)

    # Prepare socket centers once outside the loop
    # socket_centers = [(x + w // 2, y + h // 2) for x, y, w, h in socket_positions]
     # Use a set to avoid duplicate messages for the same frame
    message = None  # Ensure message is initialized to None at the start

    for ball_pos in ball_positions:
        for pocket_pos in socket_positions:
            x, y, w, h = pocket_pos
            pocket_center = (x, y)
            distance = np.linalg.norm(np.array(ball_pos[:2]) - np.array(pocket_center))
            if distance < 100:  # Proximity threshold
                # Create a message for the first detected event and stop further checks
                message = f"Ball in the socket at position: {ball_pos[:2]} in frame {i}"
                break  # Stop after the first detection
        if message:
            break  # Stop if a message has been set

    # Draw all messages in the top left corner
    if message:
        # aggregated_message = " | ".join(messages)
        frame_with_pockets = draw_text_with_background(frame_with_pockets, message)

    # Convert to RGB and save the frame
    img = cv2.cvtColor(frame_with_balls, cv2.COLOR_BGR2RGB)
    cv2.imwrite(f"results/frame_{i}.jpg", img)




ball_template_folder = 'template'
pocket_template_folder = 'sockets'
video_path = 'bilard_1.mp4'
frames = read_video(video_path)
# Get the first frame and detect socket positions
initial_frame, initial_socket_positions = get_socket_positions_from_first_frame(frames, pocket_template_folder)

if initial_frame is not None:
    executor = get_reusable_executor(max_workers=16, timeout=2)
    frame_args = [(frame, i, ball_template_folder, initial_socket_positions) for i, frame in enumerate(frames)]
    results = executor.map(lambda args: process_frame(*args), frame_args)
    results_list = list(results)  # Collecting results
else:
    print("No frames available to process.")


In [8]:
import cv2
import os
import glob

def create_video_from_images(image_folder, output_video, frame_rate=30):
    image_files = glob.glob(os.path.join(image_folder, '*.jpg'))
    image_files = sorted(image_files, key=lambda x: int(x.split("_")[1].split(".")[0]))

    first_image = cv2.imread(image_files[0])
    height, width, layers = first_image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, frame_rate, (width, height))

    for filename in image_files:
        img = cv2.imread(filename)
        out.write(img)

    out.release()
    cv2.destroyAllWindows()

create_video_from_images('results', 'movie.mp4')